In [1]:
"""
Neurosynth practice example 
2018_07_29
Kun Il Kim
"""
# ## Importing modules
"""
 Let's start with the basics. In Python, most modules (i.e., organized chunks of code) are 
inaccessible by default. Other than a few very basic built-in functions, 
you'll need to explicitly include every piece of code you want to work with. 
This may seem cumbersome, but it has the nice effect of (a) making sure you always know exactly 
what dependencies your code has, and (b) minimizing the memory footprint of your app 
by only including functionality you know you'll need.
 
 Like most Python packages, Neurosynth consists of several modules arranged into 
 a semi-sensible tree structure. For this lab, we'll need functionality available 
 in several modules, which we can include like so:
"""
from neurosynth.base.dataset import Dataset
from neurosynth.analysis import meta, decode, network
import neurosynth


C:\Users\kkim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



 ## Creating a new dataset

 Next, we create a Dataset, which is the core object most Neurosynth tools operate on. We initialize a Dataset by passing in a database file, which is essentially just a giant list of activation coordinates and associated study IDs. This file can be downloaded from the Neurosynth website or installed from the data submodule (see the Readme for instructions).
 
 Creating the object will take a few minutes on most machines, as we need to process about 200,000 activations drawn from nearly 6,000 studies. Once that's done, we also need to add some features to the Dataset. Features are just variables associated with the studies in our dataset; literally any dimension a study could be coded on can constitute a feature that Neurosynth can use. In practice, the default set of features included in the data download includes 500 psychological terms (e.g., 'language', 'emotion', 'memory', etc.) that occur with some frequency in the dataset. So when we're talking about the "emotion" feature, we're really talking about how frequently each study in the Dataset uses the word 'emotion' in the full-text of the corresponding article.
 
 Let's go ahead and create a dataset and add some features:


In [2]:
#This operation takes a long time. Only do it single time

dataset = Dataset('database.txt')
dataset.add_features('features.txt')

# Because this takes a while, we'll save our Dataset object to disk. 
#That way, the next time we want to use it, we won't have to sit through 
#the whole creation operation again:
dataset.save('dataset.pkl')

In [8]:
dataset = Dataset.load('data/dataset.pkl')   # Note the capital D in the second Dataset--load() is a class method

In [13]:
ids1 = dataset.get_studies(features='theory mind', frequency_threshold=0.05)
ids2 = dataset.get_studies(features='emotion', frequency_threshold=0.05)
ids3 = dataset.get_studies(features='valence', frequency_threshold=0.05)

In [14]:
print(len(ids1))
print(len(ids2))
print(len(ids3))



175
901
358


In [21]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\kkim\Desktop\neurosynth_project


In [17]:
# Run a meta-analysis on emotion
ids = dataset.get_studies('emo*', frequency_threshold=0.05)
ma = meta.MetaAnalysis(dataset, ids)
ma.save_results('.', 'emotion') 

ids = dataset.get_studies(expression='emo* &~ (reward* | pain*)', frequency_threshold=0.05)
ma = meta.MetaAnalysis(dataset, ids)
ma.save_results('.', 'emotion_without_reward_or_pain')
print("Found %d studies." %len(ids))

C:\Users\kkim\Anaconda3\lib\site-packages\neurosynth\analysis\meta.py:134: RuntimeWarning: invalid value encountered in true_divide
  pFgA = pAgF * pF / pA
C:\Users\kkim\Anaconda3\lib\site-packages\neurosynth\analysis\meta.py:139: RuntimeWarning: invalid value encountered in true_divide
  pFgA_prior = pAgF * prior / pAgF_prior
Generating LALR tables


Found 1711 studies.


In [20]:
ids = dataset.get_studies(expression='emotion | valence | theory mind', frequency_threshold=0.05)
ma = meta.MetaAnalysis(dataset, ids)
ma.save_results('.', 'emotion valence theory mind')
print("Found %d studies." %len(ids))

Generating LALR tables


Found 1300 studies.


In [6]:
ids = dataset.get_studies(expression='punishment', frequency_threshold=0.1)
ma = meta.MetaAnalysis(dataset, ids)
ma.save_results('.', 'punishment')
print("Found %d studies." % len(ids))

Generating LALR tables


Found 60 studies.


In [19]:
ids = dataset.get_studies(expression='(decis* | valu* | mentali*  )', frequency_threshold=0.05)
ma = meta.MetaAnalysis(dataset, ids)
ma.save_results('.', 'process_1')
print("Found %d studies." % len(ids))

Generating LALR tables


Found 1761 studies.


In [21]:
ids = dataset.get_studies(expression='(decis* | valu* | empath*  )', frequency_threshold=0.05)
ma = meta.MetaAnalysis(dataset, ids)
ma.save_results('.', 'process_2')
print("Found %d studies." % len(ids))

Generating LALR tables


Found 1824 studies.


Meta-analytic contrasts


In addition to various logical operations, one handy thing you can do with Neurosynth is perform meta-analytic contrasts. Meaning, you can identify voxels in which the average likelihood of activation being reported differ for two different sets of studies. For example, let's say you want to meta-analytically contrast studies that use the term 'recollection' with studies that use the term 'recognition'. You can do this by defining both sets of studies separately, and then passing them both to the meta-analysis object:

In [32]:
# Get the recognition studies and print some info...
cont1_ids = dataset.get_studies('self', frequency_threshold=0.05)
print("We found %d studies of self" % len(cont1_ids))

# Repeat for recollection studies
cont2_ids = dataset.get_studies('social', frequency_threshold=0.05)
print("We found %d studies of other" % len(cont2_ids))

# Run the meta-analysis
ma = meta.MetaAnalysis(dataset, cont1_ids, cont2_ids)
ma.save_results('.', 'self vs social')

We found 986 studies of self
We found 1098 studies of other


In [22]:
# Get the recognition studies and print some info...
cont1_ids = dataset.get_studies('(decis* | valu* | mentali*  )', frequency_threshold=0.05)
print("We found %d studies of cond1" % len(cont1_ids))

# Repeat for recollection studies
cont2_ids = dataset.get_studies('(decis* | valu* | empath*  )', frequency_threshold=0.05)
print("We found %d studies of cond2" % len(cont2_ids))

# Run the meta-analysis
ma = meta.MetaAnalysis(dataset, cont1_ids, cont2_ids)
ma.save_results('.', 'cond1 vs cond2')

C:\Users\kkim\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:700: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sdict = dict((k, v) for k, v in compat.iteritems(self) if k in columns)


TypeError: 'SparseDataFrame' objects are mutable, thus they cannot be hashed

In [3]:
# Get the recognition studies and print some info...
cont1_ids = dataset.get_studies('social', frequency_threshold=0.05)
print("We found %d studies of self" % len(cont1_ids))

# Repeat for recollection studies
cont2_ids = dataset.get_studies('self', frequency_threshold=0.05)
print("We found %d studies of other" % len(cont2_ids))

# Run the meta-analysis
ma = meta.MetaAnalysis(dataset, cont1_ids, cont2_ids)
ma.save_results('.', 'social-self')

We found 1098 studies of self
We found 986 studies of other


C:\Users\kkim\Anaconda3\lib\site-packages\neurosynth\analysis\meta.py:134: RuntimeWarning: invalid value encountered in true_divide
  pFgA = pAgF * pF / pA
C:\Users\kkim\Anaconda3\lib\site-packages\neurosynth\analysis\meta.py:139: RuntimeWarning: invalid value encountered in true_divide
  pFgA_prior = pAgF * prior / pAgF_prior


Seed-based coactivation maps

By now you're all familiar with seed-based functional connectivity. We can do something very similar at a meta-analytic level (e.g., Toro et al, 2008, Robinson et al, 2010, Chang et al, 2012) using the Neurosynth data. Specifically, we can define a seed region and then ask what other regions tend to be reported in studies that report activity in our seed region. The Neurosynth tools make this very easy to do. We can either pass in a mask image defining our ROI, or pass in a list of coordinates to use as the centroid of spheres. In this example, we'll do the latter:

In [6]:
# Seed-based coactivation
#network.coactivation(dataset, [[3, 49, 31]], threshold=0.1, r=10, output_dir='.', prefix='Admpfc_seed')
#network.coactivation(dataset, [[2, 24, 46]], threshold=0.1, r=10, output_dir='.', prefix='Pdmpfc_seed')
network.coactivation(dataset, [[2, 44, 12]], threshold=0.1, r=10, output_dir='.', prefix='avdmpfc_seed')
network.coactivation(dataset, [[4, 52, 8]], threshold=0.1, r=10, output_dir='.', prefix='mmpfc_seed')
network.coactivation(dataset, [[0, 56, 2]], threshold=0.1, r=10, output_dir='.', prefix='vmpfc_seed')

In [6]:
network.coactivation(dataset, [[2, 34, -6]], threshold=0.1, r=10, output_dir='.', prefix='4_1')
network.coactivation(dataset, [[2, 34, 4]], threshold=0.1, r=10, output_dir='.',prefix='2_1')
network.coactivation(dataset, [[2, 34, 14]], threshold=0.1, r=10, output_dir='.', prefix='2_2')
network.coactivation(dataset, [[2, 34, 24]], threshold=0.1, r=10, output_dir='.', prefix='12_1')
network.coactivation(dataset, [[2, 34, 34]], threshold=0.1, r=10, output_dir='.', prefix='12_2')
network.coactivation(dataset, [[2, 34, 44]], threshold=0.1, r=10, output_dir='.', prefix='11_1')
network.coactivation(dataset, [[2, 34, 54]], threshold=0.1, r=10, output_dir='.', prefix='11_2')
network.coactivation(dataset, [[2, 44, -16]], threshold=0.1, r=10, output_dir='.', prefix='4_2')
network.coactivation(dataset, [[2, 44, -6]], threshold=0.1, r=10, output_dir='.', prefix='4_3')
network.coactivation(dataset, [[2, 44, 4]], threshold=0.1, r=10, output_dir='.', prefix='2_3')
network.coactivation(dataset, [[2, 44, 14]], threshold=0.1, r=10, output_dir='.', prefix='2_4')
network.coactivation(dataset, [[2, 44, 24]], threshold=0.1, r=10, output_dir='.', prefix='12_3')
network.coactivation(dataset, [[2, 44, 34]], threshold=0.1, r=10, output_dir='.', prefix='11_3')
network.coactivation(dataset, [[2, 44, 44]], threshold=0.1, r=10, output_dir='.', prefix='11_4')
network.coactivation(dataset, [[2, 54, -6]], threshold=0.1, r=10, output_dir='.', prefix='4_4')
network.coactivation(dataset, [[2, 34, 4]], threshold=0.1, r=10, output_dir='.', prefix='2_5')
network.coactivation(dataset, [[2, 54, 14]], threshold=0.1, r=10, output_dir='.', prefix='6_1')
network.coactivation(dataset, [[2, 54, 24]], threshold=0.1, r=10, output_dir='.', prefix='6_2')
network.coactivation(dataset, [[2, 54, 34]], threshold=0.1, r=10, output_dir='.', prefix='6_3')
network.coactivation(dataset, [[2, 64, -6]], threshold=0.1, r=10, output_dir='.', prefix='4_5')
network.coactivation(dataset, [[2, 64, 4]], threshold=0.1, r=10, output_dir='.', prefix='6_4')
network.coactivation(dataset, [[2, 64, 14]], threshold=0.1, r=10, output_dir='.', prefix='6_5')
network.coactivation(dataset, [[2, 64, 24]], threshold=0.1, r=10, output_dir='.', prefix='6_6')



In [10]:
feat_keywords = ['reward', 'anticipation', 'monetary', 'responses', 'motivation', 'loss', 'incentive', 'punishment', 'gain', 'outcome',
                 'social', 'empathy', 'moral', 'person', 'judgments', 'mentalizing', 'theory', 'perspective', 'cognition', 'mind',
                 'memory', 'events', 'imagery', 'autobiographical', 'retrieval', 'episodic', 'memories', 'future', 'semantic', 'past', 'event', 
                'decision', 'choice', 'risk', 'decisions', 'uncertainty', 'risky', 'taking', 'behavior', 'preference', 'probability',
                'inhibition', 'control', 'stop', 'motor', 'trials', 'nogo', 'cognitive', 'suppression', 'aggression', 'error', 
                'pain', 'stimulation', 'somatosensory', 'intensity', 'noxious', 'heat', 'nociceptive', 'placebo', 'chronic', 'sensory']


In [2]:
feat_keywords2 = ['eye', 'gaze', 'movements', 'eyes', 'visual', 'saccades', 'saccade', 'target', 'fixation', 'direction',
                 'decision','choice', 'risk', 'decisions', 'choices', 'uncertainty', 'outcomes', 'risky', 'taking', 'outcome', 
                 'memory', 'events', 'imagery', 'autobiographical', 'retrieval', 'episodic' 'memories' 'future' 'mental' 'semantic',
                 'motor', 'movement', 'movements', 'sensorimotor', 'primary', 'finger', 'control', 'imagery', 'tasks', 'force', 
                 'social', 'empathy', 'moral', 'person' 'judgments', 'mentalizing', 'mental', 'theory', 'people', 'mind', 
                 'reward', 'anticipation', 'monetary', 'responses', 'rewards', 'motivation', 'motivational', 'loss', 'incentive', 'punishment',
                 'cues','target', 'trials', 'cue', 'switching', 'stimulus' 'targets', 'preparation', 'switch', 'selection',
                 'conflict', 'interference', 'control', 'incongruent', 'trials', 'stroop', 'congruent', 'cognitive', 'behavioral', 'rt',
                 'inhibition', 'control', 'inhibitory', 'stop' 'motor', 'trials', 'nogo', 'cognitive', 'suppression', 'aggression', 
                 'fear anxiety', 'threat', 'responses', 'conditioning', 'cs', 'extinction', 'autonomic', 'conditioned', 'arousal',
                 'memory', 'performance', 'cognitive', 'wm', 'tasks', 'verbal', 'load', 'executive', 'test', 'maintenance',
                 'pain', 'painful', 'stimulation', 'somatosensory', 'intensity', 'noxious', 'heat', 'nociceptive', 'placebo', 'chronic']

In [7]:
# Decode images

decoder = decode.Decoder(dataset, features=feat_keywords2)
data = decoder.decode(['6_6_uniformity-test_z_FDR_0.01.nii.gz',
                      '6_5_uniformity-test_z_FDR_0.01.nii.gz',
                      '6_4_uniformity-test_z_FDR_0.01.nii.gz',
                      '6_3_uniformity-test_z_FDR_0.01.nii.gz',
                      '6_2_uniformity-test_z_FDR_0.01.nii.gz',
                      '6_1_uniformity-test_z_FDR_0.01.nii.gz',
                      '2_5_uniformity-test_z_FDR_0.01.nii.gz',
                      '2_4_uniformity-test_z_FDR_0.01.nii.gz',
                      '2_3_uniformity-test_z_FDR_0.01.nii.gz',
                      '2_2_uniformity-test_z_FDR_0.01.nii.gz',
                      '2_1_uniformity-test_z_FDR_0.01.nii.gz',
                      '11_4_uniformity-test_z_FDR_0.01.nii.gz',
                      '11_3_uniformity-test_z_FDR_0.01.nii.gz',
                      '11_2_uniformity-test_z_FDR_0.01.nii.gz',
                      '11_1_uniformity-test_z_FDR_0.01.nii.gz',
                      '4_5_uniformity-test_z_FDR_0.01.nii.gz',
                      '4_4_uniformity-test_z_FDR_0.01.nii.gz',
                      '4_3_uniformity-test_z_FDR_0.01.nii.gz',
                      '4_2_uniformity-test_z_FDR_0.01.nii.gz',
                      '4_1_uniformity-test_z_FDR_0.01.nii.gz',
                      '12_1_uniformity-test_z_FDR_0.01.nii.gz',
                      '12_2_uniformity-test_z_FDR_0.01.nii.gz',
                       '12_3_uniformity-test_z_FDR_0.01.nii.gz'
                      ], save='manu_keywords_decoding_results.txt')